# 快速开始序贯（sequential）模型

In [1]:
from keras.models import Sequential
from keras.layers import Dense,Activation

Using TensorFlow backend.


In [2]:
model=Sequential([Dense(32,input_shape=(784,)),Activation('relu'),Dense(10),Activation('softmax')])
model=Sequential()
model.add(Dense(32,input_shape=(784,)))
model.add(Activation('relu))

# 编译


## 1.优化器 optimizers
优化器是编译Keras模型的必要的两个参数之一

In [3]:
from keras import optimizers
model=Sequential()
model.add(Dense(64,kernel_initializer='uniform',input_shape=(10,)))
model.add(Activation('tanh'))
model.add(Activation('softmax'))

sgd=optimizers.SGD(lr=.01,decay=1e-6,momentum=.9,nesterov=True)
model.compile(loss='mean_squared_error',optimizer=sgd)

可以在调用model.compile()之前初始化一个优化器对象，然后传入该函数（如上所示），也可以在调用时传递一个优化器名，参数为默认值。

In [8]:
#pass optimizer by name: default parameters will be used

model.compile(loss='mean_squared_error''optimizer='sgd)

### 所有优化器都可用的参数
clipnorm和clipvalue 用于对梯度进行裁剪

In [ ]:
#All parameter gradients will be clipped to
#a maximum norm of 1

sgd=optimizers.SGD(lr=.01,clipnorm=1.)

#a maximum value of 0,5 and a minimum value of -0.5
sgd=optimizers.SGD(lr=.01,clipvalue=.5)



### 1.1 SGD  
随机梯度下降法，支持动量参数，支持学习衰减率，支持Nesteror动量

In [9]:
optimizers.SGD(lr=.01,momentum=.0,decay=.0,nesterov=False)

#### 参数  
* lr:大于或等于0的浮点数，学习率
* momentum:大于或等于0的浮点数，动量参数
* decay:大或等于0的浮点数，每次更新后的学习率衰减值
* nesteror: 布尔值，确定时候使用Nesterov动量

### 1.2 RMSprop
除学习率可调整外，建议其他参数默认  
该优化器通常是面对递归神经网络的一个良好选择

In [10]:
optimizers.RMSprop(lr=.001,rho=.9,epsilon=1e-6)

#### 参数
* lr:学习率
* rho:大或等于0的浮点数
* epsilom: 大或等于0的小浮点数，防止除0错误

### 1.3 Adagrad
建议保持优化器的默认参数不变

In [11]:
optimizers.Adagrad(lr=.01,epsilon=1e-6)

#### 参数
* lr:学习率
* epsilom: 大或等于0的小浮点数，防止除0错误

### 1.4 Adadelta
建议保持优化器的默认参数不变

In [12]:
optimizers.Adadelta(lr=1.0,rho=.95,epsilon=1e-6)

#### 参数
* lr:学习率
* rho:大或等于0的浮点数
* epsilom: 大或等于0的小浮点数，防止除0错误

## 2 目标函数objectives
目标函数，或称损失函数，是编译一个模型必须的两个参数之一

In [13]:
model.compile(loss='mean_squared_error',optimizer='sgd')

可以通过传递**预定目标函数名字**指定目标函数,也可以传递一个**Theano/Tensorflow的符号函数**作为目标函数，该函数对每个数据点都应该只返回一个标量值，并以下列两个参数为参数：  
* y_true:真实的数据标签，张量  
* y_pred:预测值，与y_true相同shape的张量   

真实的优化目标函数是在各个数据点得到的损失函数之和的均值

#### 可用的目标函数

* mean_squared_error or mse
* mean_absolute_error or mae
* mean_absolute_percentage_error or mape
* mena_squared_logarithmic_error or msle
* squared_hinge
* hinge
* categorical_hinge
* binary_crossentropy(二元交叉熵，也称对数损失)
* logcosh
* categorical_crossentropy(多类交叉熵，需要将标签转化为二值序列,可使用**to_categorical**函数)
* sparse_categorical_crossentropy(如上，可接受稀疏标签)
* kullback_leibler_divergence(从预测值概率分布Q到真值概率分布P的信息增益，泳衣度量两个分布的差异)
* poisson(mean(predictions-targets*log(predictions)))
* cosine_proximity(预测值与真实值的余弦距离平均值的相反数)

## 3 性能评估

### 使用方法

性能评估模块提供了一系列用于模型性能评估的函数，这些函数在模型编译时由metrics关键字设置  
性能评估函数与目标函数类似，只不过评估结果不会用于训练  
可以通过***字符串***来使用域定义的性能评估函数

In [4]:
model.compile(loss='mse',
                       optimizer='sgd',
                          metrics=['mae','acc'])

也可以定义一个tensorflow函数并使用

In [5]:
from keras import metrics

In [6]:
model.compile(loss='mse',optimizer='sgd',metrics=[metrics.mae,metrics.categorical_accuracy])

#### 参数  
* y_true:真实标签，张量  
* y_pred:预测值，张量

#### 返回值  
单个用以代表输出各个数据点上均值的值

### 可用预定义张量  
除fbeta_score额外拥有默认参数beta=1外，其他各个性能指标的参数均为y_train和y_pred

* binary_accuracy:对二分类问题，计算在所有预测值上的平均正确率
* catagorical_accuracy:多多分类问题，计算在所有预测值上的平均正确率
* sparse_catagorical_accuracy:与上相同，对稀疏的目标值预测时有用
* top_k_catagorical_accuracy:计算top_k的正确率，当预测值的前k个值中存在目标类别即认为预测正确
* sparse_top_k_catagorical_accuracy:与上相同，适用于稀疏情况

### 定制评估函数  
定制的评估函数可以在模型编译时传入，该函数应该以(y_true,y_pred)为参数，返回单个张量，或从metric_name映射到metric_value的字典，下面是一个示例：

In [7]:
import keras.backend as K
def mean_pred(y_true,y_pred):
    return K.mean(y_pred)

model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy',mean_pred])

# 训练

keras以numpy数组作为输入数据和标签的数据类型，训练模型一般使用fit函数。

In [ ]:
fit(self,x,y,batch_size=32,epoch=10,verbose=1,callbacks=None,validation_split=.0,
      validation_data=None,shuffle=True,class_weight=None,sample_weight=None,initial_epoch=0)

本函数将模型训练n_epoch轮，其参数有：  
* x:输入数据。如果模型只有一个输入，x的类型就是numpy array，如果模型有多个输入，那么x的类型应该为List,list的元素是对应于各个输入的numpy array
* y:标签，numpy array
* batch_size: 整数，指定进行梯度下降时每个Batch包含的样本数，训练时一个batch的样本会被计算一次梯度下降，是目标函数优化一步。
* epochs:整数，训练终止时的epoch值，训练将在达到epoch值时停止，当没有设置initial_epoch时，就是训练的总轮数，否则训练的总轮数为epochs-inital_epoch  
* verbose:日志显示，0为不在标注输出流输出日志信息，1为输出进度条记录，2为每个epoch输出一行数据  
* callbacks:list, 其中元素是keras.callbacks.Callback的对象，这个list中的回调函数将会在训练过程中的适当时机被调用  
* validation_split: 0-1中的浮点数，用来指定训练集中一定比例数据作为验证集，验证集将不参与训练，并在每个epoch结束后测试的模型指标，如损失函数，精确度等。注意：validation_split的划分在shuffle以前，因此如果你的数据本身是有序的，需要先手工打乱再指定validation_split，否则可能会使验证集样本不均匀  
* validation_data: 形式为(X,y)的tuple，指定的验证集，覆盖vaolidation_split
* shuffle: 布尔值或字符串，表示训练过程是否需要打乱样本的顺序。
* class_weight: 权值的numpy array，用于在训练时调整损失函数(仅用于训练)。
* initial_epoch: 从该参数指定的epoch开始训练，在继续之前的训练时有用

***fit函数返回一个history的对象，其History.history属性记录了损失函数和其他指标的数值随epoch变化的情况，如果有验证集，也包含了验证集的指标变化***

#### 例子

In [16]:
# for a single_input model with 2 classes(binary classification)
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=100))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',
                           metrics=['accuracy'] )
# generate dummy data
import numpy as np
data=np.random.random((1000,100))
labels=np.random.randint(2,size=1000)

#trian the model,interating on the data in batches of 32 samples
model.fit(data,labels,epochs=20,batch_size=32)

Epoch 1/20
1000/1000 [==============================] - 1s 580us/step - loss: 0.7119 - acc: 0.4840
Epoch 2/20
1000/1000 [==============================] - 0s 146us/step - loss: 0.6978 - acc: 0.5080
Epoch 3/20
1000/1000 [==============================] - 0s 140us/step - loss: 0.6906 - acc: 0.5470
Epoch 4/20
1000/1000 [==============================] - 0s 138us/step - loss: 0.6880 - acc: 0.5440
Epoch 5/20
1000/1000 [==============================] - 0s 132us/step - loss: 0.6818 - acc: 0.5580
Epoch 6/20
1000/1000 [==============================] - 0s 132us/step - loss: 0.6799 - acc: 0.5680
Epoch 7/20
1000/1000 [==============================] - 0s 141us/step - loss: 0.6743 - acc: 0.5840
Epoch 8/20
1000/1000 [==============================] - 0s 141us/step - loss: 0.6690 - acc: 0.6050
Epoch 9/20
1000/1000 [==============================] - 0s 145us/step - loss: 0.6711 - acc: 0.5790
Epoch 10/20
1000/1000 [==============================] - 0s 133us/step - loss: 0.6642 - acc: 0.5970
Epoch 11/

In [5]:
# for a single_input model with 10 classes(catagorical classification)
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=100))
model.add(Dense(10,activation='softmax'))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',
                           metrics=['accuracy'] )
# generate dummy data
import numpy as np
data=np.random.random((1000,100))
labels=np.random.randint(10,size=1000)

#convert labels to categorical one_hot encoding
import keras
one_hot_labels=keras.utils.to_categorical(labels,num_classes=10)
#trian the model,interating on the data in batches of 32 samples
model.fit(data,one_hot_labels,epochs=10,batch_size=32)

Epoch 1/10
1000/1000 [==============================] - 2s 2ms/step - loss: 2.3593 - acc: 0.1040
Epoch 2/10
1000/1000 [==============================] - 0s 157us/step - loss: 2.3236 - acc: 0.1070
Epoch 3/10
1000/1000 [==============================] - 0s 157us/step - loss: 2.3098 - acc: 0.1230
Epoch 4/10
1000/1000 [==============================] - 0s 171us/step - loss: 2.3014 - acc: 0.1420
Epoch 5/10
1000/1000 [==============================] - 0s 191us/step - loss: 2.2913 - acc: 0.1390
Epoch 6/10
1000/1000 [==============================] - 0s 155us/step - loss: 2.2845 - acc: 0.1340
Epoch 7/10
1000/1000 [==============================] - 0s 164us/step - loss: 2.2774 - acc: 0.1550
Epoch 8/10
1000/1000 [==============================] - 0s 180us/step - loss: 2.2709 - acc: 0.1480
Epoch 9/10
1000/1000 [==============================] - 0s 177us/step - loss: 2.2615 - acc: 0.1580
Epoch 10/10
1000/1000 [==============================] - 0s 211us/step - loss: 2.2551 - acc: 0.1690


## 例子

在keras 代码包的examples文件夹中，可以找到使用真实数据的示例模型：
* CIFAR10小图片分类：使用cnn和实时数据提升
* IMDB电影评价观点分类：使用LSTM处理成序列的词语
* Reuters（路透社）新闻主题分类：使用多层感知器(MLP)
* MNIST手写数据识别：使用多层感知器和cnn
* 字符集文本生成：使用LSTM

#### 基于多层感知器的softmax多分类

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

# Generate dummy data
import numpy as np
x_train = np.random.random((1000, 20))
y_train = keras.utils.to_categorical(np.random.randint(10, size=(1000, 1)), num_classes=10)
x_test = np.random.random((100, 20))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)

model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(64, activation='relu', input_dim=20))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=20,
          batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)

Epoch 1/20
1000/1000 [==============================] - 2s 2ms/step - loss: 2.4383 - acc: 0.1060
Epoch 2/20
1000/1000 [==============================] - 0s 57us/step - loss: 2.3631 - acc: 0.1090
Epoch 3/20
1000/1000 [==============================] - 0s 50us/step - loss: 2.3356 - acc: 0.1210
Epoch 4/20
1000/1000 [==============================] - 0s 42us/step - loss: 2.3295 - acc: 0.1140
Epoch 5/20
1000/1000 [==============================] - 0s 58us/step - loss: 2.3197 - acc: 0.1090
Epoch 6/20
1000/1000 [==============================] - 0s 45us/step - loss: 2.3341 - acc: 0.0990
Epoch 7/20
1000/1000 [==============================] - 0s 50us/step - loss: 2.3143 - acc: 0.0990
Epoch 8/20
1000/1000 [==============================] - 0s 57us/step - loss: 2.3061 - acc: 0.1190
Epoch 9/20
1000/1000 [==============================] - 0s 52us/step - loss: 2.3117 - acc: 0.1170
Epoch 10/20
1000/1000 [==============================] - 0s 47us/step - loss: 2.3067 - acc: 0.1110
Epoch 11/20
1000/100

#### MLP的二分类

In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Generate dummy data
x_train = np.random.random((1000, 20))
y_train = np.random.randint(2, size=(1000, 1))
x_test = np.random.random((100, 20))
y_test = np.random.randint(2, size=(100, 1))

model = Sequential()
model.add(Dense(64, input_dim=20, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          epochs=20,
          batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)

Epoch 1/20
1000/1000 [==============================] - 1s 559us/step - loss: 0.7365 - acc: 0.4730
Epoch 2/20
1000/1000 [==============================] - 0s 57us/step - loss: 0.7139 - acc: 0.4970
Epoch 3/20
1000/1000 [==============================] - 0s 49us/step - loss: 0.6929 - acc: 0.5490
Epoch 4/20
1000/1000 [==============================] - 0s 54us/step - loss: 0.7053 - acc: 0.4950
Epoch 5/20
1000/1000 [==============================] - 0s 53us/step - loss: 0.7015 - acc: 0.5060
Epoch 6/20
1000/1000 [==============================] - 0s 51us/step - loss: 0.7010 - acc: 0.5250
Epoch 7/20
1000/1000 [==============================] - 0s 44us/step - loss: 0.6942 - acc: 0.5210
Epoch 8/20
1000/1000 [==============================] - 0s 52us/step - loss: 0.6890 - acc: 0.5250
Epoch 9/20
1000/1000 [==============================] - 0s 52us/step - loss: 0.6985 - acc: 0.5110
Epoch 10/20
1000/1000 [==============================] - 0s 42us/step - loss: 0.6929 - acc: 0.5320
Epoch 11/20
1000/1

类似VGG的卷积神经网络

In [4]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

# Generate dummy data
x_train = np.random.random((100, 100, 100, 3))
y_train = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)
x_test = np.random.random((20, 100, 100, 3))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(20, 1)), num_classes=10)

model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

model.fit(x_train, y_train, batch_size=32, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=32)

Epoch 1/10
100/100 [==============================] - 3s 32ms/step - loss: 2.3788
Epoch 2/10
100/100 [==============================] - 0s 2ms/step - loss: 2.3373
Epoch 3/10
100/100 [==============================] - 0s 2ms/step - loss: 2.2930
Epoch 4/10
100/100 [==============================] - 0s 2ms/step - loss: 2.2912
Epoch 5/10
100/100 [==============================] - 0s 2ms/step - loss: 2.2828
Epoch 6/10
100/100 [==============================] - 0s 2ms/step - loss: 2.3000
Epoch 7/10
100/100 [==============================] - 0s 2ms/step - loss: 2.2897
Epoch 8/10
100/100 [==============================] - 0s 2ms/step - loss: 2.2890
Epoch 9/10
100/100 [==============================] - 0s 2ms/step - loss: 2.2761
Epoch 10/10
20/20 [==============================] - 0s 5ms/step


#### 使用LSTM的序列分类

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(max_features, output_dim=256))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=16, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=16)

#### 使用1D卷积的序列分类

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=(seq_length, 100)))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=16, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=16)

#### 用于序列分类的栈式LSTM

![](0.png)

在该模型中，将**三个LSTM**堆叠在一起，使得该模型能够学习**更高层次的时域特征**表示  
前2层LSTM返回其全部输出序列，第三层只返回其输出序列的最后一步结果，从而使得时域维度降低（即将输入序列转换为单个向量）

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

data_dim = 16
timesteps = 8
num_classes = 10

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(32, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32))  # return a single vector of dimension 32
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Generate dummy training data
x_train = np.random.random((1000, timesteps, data_dim))
y_train = np.random.random((1000, num_classes))

# Generate dummy validation data
x_val = np.random.random((100, timesteps, data_dim))
y_val = np.random.random((100, num_classes))

model.fit(x_train, y_train,
          batch_size=64, epochs=5,
          validation_data=(x_val, y_val))

### 采用stateful LSTM的相同模型

stateful LSTM的特点是，在处理一个batch后，其内部状态（记忆）会被作为下一个batch的初始状态。状态LSTM使得我们可以在合理的计算复杂度内处理长序列。

In [1]:
from keras.utils.visualize_util import plot

answer = Sequential() 
answer.add(Merge([response, question_encoder], mode=’concat’, concat_axis=-1)) 
answer.add(LSTM(32)) 
answer.add(Dropout(0.3)) 
answer.add(Dense(vocab_size)) 
answer.add(Activation(‘softmax’))

plot(answer, to_file=’answer_model.png’)

SyntaxError: invalid character in identifier (<ipython-input-1-91ff3bfba7d9>, line 4)